In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
import glob
import pylinac as pl

In [4]:
# Find all the dicom images using glob 
# Load the images into a list
# Create a 3D array of the images

import os

# 0.4mm_L
# 0.4_Qr40_L
# 0.4_Qr76_L
# Pelvic_2_0.4_Qr40_L
# Pelvic_2_0.4_Qr76_L
# Pelvic_3_0.4_Qr76_L
# Pelvic_32_0.4_Qr40_L


images = np.load('/media/jericho/T7/sysiphus_drive/MV_CBCT/recon_kV_full.npy')

# load the fastmc pickle file 
import pickle
with open('first_layer_360.pkl','rb') as f:
    data = pickle.load(f)

images2 = data.img

In [41]:
import matplotlib.pyplot as plt
import numpy as np

roi_names = ['Seniometry Module']

def select_regions(image, num_clicks=4):
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(image, cmap='bone')#,vmin=-100,vmax=100)
    ax.set_aspect(0.25)
    ax.axis('off')

    x_coords = []
    def onclick(event):
        if len(x_coords) >= num_clicks:
            return
        ix = int(event.xdata)
        x_coords.append(ix)
        print(len(x_coords))
        plt.axvline(x=ix, color='r', linestyle='--')
        plt.text(ix, 0, roi_names[len(x_coords)-1], color='red', fontsize=12, va='bottom', ha='center')
        fig.canvas.draw()

    # Add a colorbar
    cbar = plt.colorbar(ax.imshow(image, aspect=1, cmap='bone'),orientation='horizontal')
    cbar.set_label('Pixel Value HU')

    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    plt.show()
    return x_coords

# Use the function
x_coords = select_regions(images2.T[256])
x_coords2 = select_regions(images.T[256])

1
1


In [43]:
def select_regions(image, radius=10, num_regions=6,LC=False):
    fig, ax = plt.subplots(figsize=[5,5],dpi=300)
    ax.imshow(image, cmap='gray')
    # Make the plot long and skinny
    # ax.set_aspect(0.5)
    ax.axis('off')
    mask = np.zeros_like(image)

    conc = ['PMP', 'LDPE','Poly','Acrylic','Teflon'][-1::-1]
    if LC:
        ROI_names = [str(x) for x in conc[:num_regions]]
    else:
        ROI_names = [str(x) for x in conc[num_regions:]]

    coords = []

    def circular_mask(x, y, radius, shape):
        xx, yy = np.ogrid[:shape[0], :shape[1]]
        circle = (xx - x) ** 2 + (yy - y) ** 2
        mask = circle < radius ** 2
        return mask

    def onclick(event):
        if len(coords) >= num_regions:
            return
        ix, iy = int(event.xdata), int(event.ydata)
        coords.append((ix, iy))

        circle = plt.Circle((ix, iy), radius, color='cornflowerblue', fill=False)
        
        ax.add_artist(circle)
        # Add text to label the circles
        plt.text(ix, iy, ROI_names[len(coords)-1], color='indianred', fontsize=12, va='bottom', ha='center',fontdict={'family':'serif'})

        mask[circular_mask(iy, ix, radius, image.shape)] = len(coords)
        # mask[iy-radius:iy+radius, ix-radius:ix+radius] = len(coords)
        fig.canvas.draw()

    # cbar = plt.colorbar(ax.imshow(image, aspect=1, cmap='gray'),orientation='horizontal')
    # cbar.set_label('Pixel Value HU')
    cid = fig.canvas.mpl_connect('button_press_event', onclick)
    # plt.tight_layout()
    plt.show()

    return mask

masks= []

for ii, x_coord in enumerate(x_coords2):

    if ii in [0,2]:
        LC = True
    else:
        LC = False
    # Use the function
    masks.append(select_regions(images[x_coord-10:x_coord+10].mean(axis=0),LC=LC))
    # pause until the figure is closed

masks2= []

for ii, x_coord in enumerate(x_coords):

    if ii in [0,2]:
        LC = True
    else:
        LC = False
    # Use the function
    crop=60
    masks2.append(select_regions(images2[x_coord-10:x_coord+10].mean(axis=0)[crop:-crop,crop:-crop],LC=LC))
    # pause until the figure is closed


In [ ]:

# plot the regions of interest on the images as two subplots side by side 
fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].imshow(images2.T[256], cmap='bone')

for ii, mask in enumerate(masks2):
    ax[0].imshow(mask,alpha=0.5)
    # label the regions in the center of the mask values
    for i in range(1,mask.max()+1):
        y,x = np.where(mask==i)
        ax[0].text(x.mean(),y.mean(),roi_names[ii],color='red',fontsize=12,va='center',ha='center')
    

In [36]:
plt.figure()
plt.imshow(mask)

In [34]:
ax1.imshow(images[0])

In [11]:
# Generate 4 subplots showing the four masks overlayed on the image using nans for zero in the masks
fig, ax = plt.subplots(2,2, figsize=[8,8])
ax = ax.ravel()
titles = ['Ag low conc.', 'Ag high conc.', 'Ag-Bi low conc.', 'Ag-Bi high conc.']
for i, (mask,x_coord) in enumerate(zip(masks,x_coords)):
    image = images2[x_coord-40:x_coord+40].mean(axis=0)
    mask_temp = mask.copy()
    mask_temp[mask==0] = np.nan
    ax[i].imshow(image, aspect='auto', cmap='turbo',vmin=5,vmax=25)
    ax[i].imshow(mask_temp, aspect='auto', cmap='tab20',vmin=0,vmax=5)
    ax[i].axis('off')
    ax[i].set_title(titles[i])

for i, (mask,x_coord) in enumerate(zip(masks,x_coords2)):
    image = images[x_coord-40:x_coord+40].mean(axis=0)
    mask_temp = mask.copy()
    mask_temp[mask==0] = np.nan
    ax[i].imshow(image, aspect='auto', cmap='turbo',vmin=5,vmax=25)
    ax[i].imshow(mask_temp, aspect='auto', cmap='tab20',vmin=0,vmax=5)
    ax[i].axis('off')
    ax[i].set_title(titles[i])

In [66]:
# Calculate the mean pixel value for each mask
mean_values = []
std_values = []

for x_coord, mask in zip(x_coords, masks):
    mean_values.append([])
    for integer in np.unique(mask):
        if integer == 0:
            continue
        temp_mean = []
        for index in range(x_coord-10, x_coord+10):
            temp_mean.append(images2[index][mask == integer].mean())
        mean_values[-1].append(np.mean(temp_mean))
        std_values.append(np.std(temp_mean))

# Plot the mean values with error bars
fig, ax = plt.subplots(dpi=300,figsize=[6,4])
colors = ['cornflowerblue', 'coral', 'k', 'coral']
conc = np.array([0,0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.6,1,1.6,1.8])

import colorsys
import matplotlib

def adjust_color_lightness(color, amount=0.5):
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

# Use the function
titles = ['Ag-Bi','Ag']
for ii, (mean, std) in enumerate(zip(mean_values, std_values)):
    mean_both = mean_values[ii*2] + mean_values[ii*2+ 1]
    std_both = std_values[ii*2] + std_values[ii*2+ 1]
    darker_color = adjust_color_lightness(colors[ii], 0.75)  # Darker red
    lighter_color = adjust_color_lightness(colors[ii], 1.25)  # Lighter red
    ax.errorbar(conc+0.01*(ii-1), mean_both, yerr=std_both, fmt='*', color=darker_color, ecolor=lighter_color,capsize=5, capthick=2)
    # Add a trend line
    z = np.polyfit(conc+0.01*(ii-1), mean_both, 1)
    p = np.poly1d(z)
    ax.plot(conc+0.01*(ii-1), p(conc+0.01*(ii-1)), "--", color=colors[ii],label=titles[ii])

# plt.colorbar()
plt.xlabel('Concentration (mg/mL)')
plt.ylabel('Mean Pixel Value (HU)')
plt.legend()
plt.title('Mean HU vs Conc. 20 slices 120kVp')
plt.tight_layout()

# Save the figure
# plt.savefig('20_slices_120keV_dect.png',dpi=300)

IndexError: list index out of range

In [67]:
mean_values, std_values

([[0.031455345,
   0.02277588,
   0.019604241,
   0.017235717,
   0.016481167,
   0.015238429],
  [0.03153068,
   0.022864198,
   0.01959204,
   0.017233554,
   0.016496198,
   0.015189643]],
 [1.9044328e-05,
  2.904503e-05,
  2.4358447e-05,
  1.9826515e-05,
  1.9765648e-05,
  2.0635567e-05,
  2.7947734e-05,
  1.7612454e-05,
  2.8339178e-05,
  2.2621747e-05,
  2.7190661e-05,
  1.9389565e-05])

In [3]:
import numpy as np

mean_values_exp = np.array([[0.034148104, 0.021402083, 0.01891603, 0.018038664, 0.01657083],
  [0.034020983, 0.021427434, 0.018878074, 0.018025642, 0.016601142]])

std_values_exp = np.array([6.340513e-05, 6.512419e-05, 7.264979e-05, 6.3021194e-05, 7.8659315e-05, 6.0385766e-05, 0.00013820331, 0.00010213418, 5.5717774e-05, 6.996347e-05, 5.6306344e-05, 3.848605e-05])

In [4]:


mean_values_fastmc = np.array([[0.031447463, 0.019604241, 0.017236225, 0.016477404, 0.015212287],
  [0.031498723, 0.01959204, 0.017233554, 0.016472554, 0.015189643]])

std_values_fastmc = np.array([1.7849869e-05, 2.7534854e-05, 2.4358447e-05, 2.1794356e-05, 2.035973e-05, 2.0311245e-05, 2.5390094e-05, 2.1565795e-05, 2.8339178e-05, 2.2621747e-05, 2.1836842e-05, 1.9389565e-05])

In [5]:
# compare the mean values
fig, ax = plt.subplots(dpi=300,figsize=[6,4])
colors = ['cornflowerblue', 'coral', 'k', 'coral']
# plt.plot(mean_values_exp,mean_values_fastmc,'*',label='Experiments')
# plot the mean with the error bar being the standard deviation 
point_labels = ['PMP','LDPE','Polystyrene','Acrylic','Teflon'][-1::-1]
# label each point with the material
for i in range(mean_values_exp.shape[1]):
    ax.errorbar(mean_values_exp.mean(axis=0)[i], mean_values_fastmc.mean(axis=0)[i], yerr=mean_values_fastmc.std(axis=0)[i], xerr=mean_values_exp.std(axis=0)[i], fmt='*', color='darkgray', ecolor='coral',capsize=2, capthick=2)
    plt.text(mean_values_exp.mean(axis=0)[i], mean_values_fastmc.mean(axis=0)[i], point_labels[i], fontsize=10, fontdict={'color':'indianred','family':'serif'})
# plt.errorbar(mean_values_exp.mean(axis=0), mean_values_fastmc.mean(axis=0), yerr=mean_values_fastmc.std(axis=0), xerr=mean_values_exp.std(axis=0), fmt='*', color='forestgreen', ecolor='coral',capsize=2, capthick=2)

# plot a linear fit to the data
z = np.polyfit(mean_values_exp.mean(axis=0), mean_values_fastmc.mean(axis=0), 1)
p = np.poly1d(z)
plt.plot(mean_values_exp.mean(axis=0), p(mean_values_exp.mean(axis=0)), "--", color='cornflowerblue',label='Trend line')
# plot the fit values on the plot
# plt.text(0.017, 0.0165, f'y = {z[0]:.3f}x + {z[1]:.3f}', fontsize=12)
plt.xlabel('Mean HU Experiments')
plt.ylabel('Mean HU FastMC')
plt.legend()

libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [13]:
print(mean_values_fastmc_HU, mean_values_exp_HU)
np.mean(np.abs(np.array(mean_values_exp_all) - mean_values_fastmc_all))

-185.65740967659394 -187.31037976219932


5.714565611478929

In [8]:
# compare the mean values
fig, ax = plt.subplots(dpi=300,figsize=[6,4])
colors = ['cornflowerblue', 'coral', 'k', 'coral']
# plt.plot(mean_values_exp,mean_values_fastmc,'*',label='Experiments')
# plot the mean with the error bar being the standard deviation 
point_labels = ['PMP','LDPE','Polystyrene','Acrylic','Teflon'][-1::-1]
# label each point with the material
mean_values_exp_all = []
mean_values_fastmc_all = []

attenuation_coefficient_water = 0.018878074+0.0005
attenuation_coefficient_water_fmc = 0.017233554 + 0.0005
for i in range(len(mean_values_exp[0])):
    mean_values_exp_HU = (mean_values_exp.mean(axis=0)[i] / attenuation_coefficient_water) * 1300 -1300
    mean_values_fastmc_HU = (mean_values_fastmc.mean(axis=0)[i] / attenuation_coefficient_water_fmc) * 1300 - 1300
    ax.errorbar(mean_values_exp_HU, mean_values_fastmc_HU, yerr=mean_values_fastmc.std(axis=0)[i], xerr=mean_values_exp.std(axis=0)[i], fmt='*', color='darkgray', ecolor='coral',capsize=2, capthick=2)
    plt.text(mean_values_exp_HU, mean_values_fastmc_HU, point_labels[i], fontsize=10, fontdict={'color':'indianred','family':'serif'})

    mean_values_exp_all.append(mean_values_exp_HU)
    mean_values_fastmc_all.append(mean_values_fastmc_HU)

# plot a linear fit to the data
z = np.polyfit(mean_values_exp_all, mean_values_fastmc_all, 1)
p = np.poly1d(z)
plt.plot(mean_values_exp_all, p(mean_values_exp_all), "--", color='cornflowerblue',label='Linear fit')
# plot the fit values on the plot
plt.grid('on')
# plot the r squared value on the plot using the equation from wikipedia
# https://en.wikipedia.org/wiki/Coefficient_of_determination
y = mean_values_fastmc_all
y_hat = p(mean_values_exp_all)
y_bar = np.mean(y)
ss_tot = np.sum((y - y_bar)**2)
ss_res = np.sum((y - y_hat)**2)
r_squared = 1 - (ss_res/ss_tot)

plt.text(500,0, f'R$^2$ = {r_squared:.3f}', fontsize=12)
plt.xlabel('Mean HU Experiments')
plt.ylabel('Mean HU DFCIsim')
plt.legend()

plt.tight_layout()
# Save the figure
plt.savefig('HU_comparison_dfci_Sim.png',dpi=300)

In [97]:
len(mean_values_exp[0])

6

In [45]:
# Calculate the mean pixel value for each mask
mean_values = []
std_values = []

for x_coord, mask in zip(x_coords, masks):
    mean_values.append([])
    for integer in np.unique(mask):
        if integer == 0:
            continue
        temp_mean = []
        for index in range(x_coord-10, x_coord+10):
            temp_mean.append(images[index][mask == integer].mean())
        mean_values[-1].append(np.mean(temp_mean))
        std_values.append(np.std(temp_mean))

# Plot the mean values with error bars
fig, ax = plt.subplots(dpi=300,figsize=[6,4])
colors = [ 'darkorange', 'gray']
conc = np.array([0,0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.6,1])

import colorsys
import matplotlib

def adjust_color_lightness(color, amount=0.5):
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

# Use the function
titles = ['Ag', 'Ag-Bi']
for ii, (mean, std) in enumerate(zip(mean_values[::2], std_values[::2])):
    mean_both = mean_values[ii*2] + mean_values[ii*2+ 1]
    std_both = std_values[ii*2] + std_values[ii*2+ 1]
    darker_color = adjust_color_lightness(colors[ii], 0.75)  # Darker red
    lighter_color = adjust_color_lightness(colors[ii], 1.25)  # Lighter red
    ax.errorbar(conc+0.01*(ii-1), mean_both, yerr=std_both, fmt='*', color=darker_color, ecolor=lighter_color,capsize=5, capthick=2)
    # Add a trend line
    z = np.polyfit(conc+0.01*(ii-1), mean_both, 1)
    p = np.poly1d(z)
    ax.plot(conc+0.01*(ii-1), p(conc+0.01*(ii-1)), "--", color=colors[ii],label=titles[ii])

# plt.colorbar()
plt.xlabel('Concentration (mg/mL)')
plt.ylabel('Mean Pixel Value (HU)')
plt.legend()
plt.title('Mean HU vs Conc. 20 slices 80kVp')
plt.tight_layout()

# Save the figure
plt.savefig('20_slices_80kVp_dect.png', dpi=300)

In [48]:
import pickle

with open('plot_data_120keV.pkl', 'wb') as f:
    pickle.dump((mean_values, std_values, conc, colors, titles), f)

In [91]:
plt.rcParams['text.usetex'] = False


In [49]:
W = 100
L = 25

fig = plt.figure(figsize=(13, 7))
plt.subplot(131)
cc= pli.hyperslicer(images,autoscale_cmap=False,cmap='bone',vmin=L-W/2,vmax=L+W/2)
plt.axis('off')
plt.title(f'20-70 keV W/L {W}/{L}')
plt.colorbar(orientation='horizontal')
plt.subplot(132)
c2 = pli.hyperslicer(images2,autoscale_cmap=False,cmap='bone',controls=cc,vmin=L-W/2,vmax=L+W/2)
plt.axis('off')
plt.title(f'70-140 keV W/L {W}/{L}')
plt.colorbar(orientation='horizontal')
plt.subplot(133)
c3 = pli.hyperslicer(images - images2,autoscale_cmap=False,cmap='bwr',controls=cc,vmin=-W/4,vmax=W/4)
plt.axis('off')
plt.title('Difference [HU]')
plt.colorbar(orientation='horizontal')
plt.tight_layout()

In [55]:
controls.save_animation('low_high_dect_bi_difference.gif',fig,'param1',interval=20)

In [31]:
# plt.figure()
# pli.hyperslicer(images,autoscale_cmap=False,cmap='turbo')

AttributeError: 'Controls' object has no attribute 'vbox'

In [53]:
import mpl_interactions.ipyplot as iplt

slice = x_coords[1] #1080 #253 #787 #342 #1000

nslice = 10

x = np.linspace(0, np.pi, 200)
y = np.linspace(0, 10, 200)
X, Y = np.meshgrid(x, y)

im1 = np.array(images1[slice-nslice:slice+nslice]).mean(axis=0)
im2 = np.array(images[slice-nslice:slice+nslice]).mean(axis=0)

# im1 = im1 - im1.min()
# im1 = im1 / im1.max()

# im2 = im2 - im2.min()
# im2 = im2 / im2.max()

def f(param1):
    return im1* param1 - im2

fig, ax = plt.subplots(figsize=(8,8))
controls = iplt.imshow(f, param1=(0, 2),vmin_vmax=("r", -200, 200),cmap='turbo')
plt.axis('off')
plt.colorbar(orientation='horizontal')

with controls:
    # directly using string formatting
    # the formatting is performed in the update
    iplt.title(title="120keV *{param1:.2f} - 80keV")


In [76]:
from mpl_interactions import image_segmenter
segmenter = image_segmenter(im2, mask_colors="red", mask_alpha=0.76, figsize=(7, 7))
display(segmenter)
plt.show()

In [79]:
diffs = []

for param1 in np.linspace(0, 2, 100):
    im_temp = f(param1)
    mean_low = im_temp[mask_water==1]
    mean_high = im_temp[mask_high_conc_ag==1]
    diffs.append(np.abs(mean_high.mean() - mean_low.mean())/np.sqrt(mean_high.var() + mean_low.var()))

In [81]:
plt.figure()
plt.plot(np.linspace(0, 2, 100), diffs)
plt.xlabel('weighting of 70-140keV')
plt.ylabel('CNR')
plt.title('CNR 1.0 mg/ml AG (70-140keV*weight - 20-70keV)')

Text(0.5, 1.0, 'CNR 1.0 mg/ml AG (70-140keV*weight - 20-70keV)')

In [78]:
# np.save('mask_low_conc.npy', mask_low_conc)
# np.save('mask_high_conc.npy', mask_high_conc)
# np.save('mask_water.npy', mask_water)
# np.save('mask_high_conc_ag.npy', mask_high_conc_ag)

In [77]:
mask_high_conc_ag = segmenter.mask

In [75]:
total_mask = mask_water + 2*mask_low_conc + 3 *mask_high_conc
total_mask[total_mask==0] = np.NaN

plt.figure(dpi=400)
plt.imshow(im2,cmap='bone',vmin = np.mean(im2[mask_water==1]) - 2*np.std(im2[mask_water==1]), vmax = np.mean(im2[mask_high_conc==1]) + 2*np.std(im2[mask_water==1]))
plt.imshow(total_mask, cmap='flag',alpha=0.5)
plt.axis('off')
plt.title('Image Segments')

Text(0.5, 1.0, 'Image Segments')